In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings(action='ignore')

from math import atan, sqrt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import joblib


import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Lambda, Input, Dense, Reshape, Dropout, RepeatVector, LSTM, TimeDistributed, BatchNormalization, Bidirectional
# import tensorflow_addons as tfa
from keras.utils import to_categorical
from keras import backend as K 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

from transforms3d.axangles import axangle2mat

In [ ]:
# LSTM Layer 구성

def LSTM_model(input_shape, classes):
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer = tf.keras.layers.Input(input_shape)
#     gru1 = L.LSTM(128, return_sequences = True, dropout = 0.5,kernel_regularizer=tf.keras.regularizers.L1(0.05))(input_layer)
    gru1 = L.LSTM(128, return_sequences = True, dropout = 0.3)(input_layer)
    mp = L.MaxPool1D(padding='same')(gru1)
    ap = L.AveragePooling1D(padding='same')(gru1)
    concat1 = L.Concatenate()([mp, ap])
    
    gru2 = L.LSTM(64, return_sequences = True, dropout = 0.2)(concat1)
    mp2 = L.MaxPool1D(padding='same')(gru2)
    ap2 = L.AveragePooling1D(padding='same')(gru2)
    concat2 = L.Concatenate()([mp2, ap2])
    
    gru3 = L.LSTM(64, return_sequences = True)(concat2)
    GAP = L.GlobalAveragePooling1D()(gru3)

    dense = L.Dense(classes, activation = "softmax")(GAP)
    
    model = tf.keras.models.Model(input_layer, dense)
    
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer = optimizer, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
# Train_data Shape 변환

train_data = train_sc.iloc[:,1:]
train_data = np.array(train_data)
train_data = train_data.reshape(-1,10,9)
train_data.shape

In [ ]:
# Train_data Label Shape 변환

y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

In [ ]:
# Kfold 를 이용해서 검증 데이터를 옮겨가면서 지속적인 학습을 진행 
skf = StratifiedKFold(n_splits = 5, random_state = 2048, shuffle = True)

# keras의 콜백함수인 ReduceLROnPlateau는 학습률이 개선되지 않을 때, 
# 학습률을 동적으로 조정하여 학습률을 개선하는 효과를 기대할 수 있습니다.
reLR = ReduceLROnPlateau(monitor = 'loss', patience = 5,verbose = 1,factor = 0.8) 

# loss값이 개선되지 않는 일정 점에서 동작을 정지하여 리소스 확보를 돕는다.
es = EarlyStopping(monitor='loss', patience=10, mode='min')

window = 10
columns = 9

accuracy = []
losss = []
models = []

# fold 에 따른 각각의 값 loss, accuracy, model 을 저장하기 위한 list 선언
for i, (train, validation) in enumerate(skf.split(train_data, y.argmax(1))) :
    
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)

    model = LSTM_model((window,columns), 10)

    history = model.fit(train_data[train], y[train],
                        epochs = 300, 
                        verbose=0,
                        batch_size=64,
                        callbacks=[es, reLR])
    
    k_accuracy = '%.4f' % (model.evaluate(train_data[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(train_data[validation], y[validation])[0])
    

    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

# fold 진행하고 나서 나온 값을 위에 선언한 List에 appand한다. 

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

In [ ]:
# 전체 평균 정확도와 손실값 출력
print(sum([float(i) for i in accuracy])/5)
print()
print(sum([float(i) for i in losss])/5)


In [ ]:
preds = []
for model in models:
    pred = model.predict(x_test)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred_df = pd.DataFrame(pred)

In [ ]:
temp_list = []
for i in range(len(pred_df)):
  temp_list.append(pred_df.iloc[i].argmax())

result_df = pd.DataFrame(temp_list)

In [ ]:
result_df